### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [1]:
# write your code from here
try:
    import avro.schema
    import io
    from avro.io import DatumWriter, BinaryEncoder
except ImportError:
    import sys
    !{sys.executable} -m pip install avro-python3
    import avro.schema
    import io
    from avro.io import DatumWriter, BinaryEncoder

import json
import pandas as pd

# Step 2: Define Avro schema
schema_json = {
    "namespace": "example.avro",
    "type": "record",
    "name": "Customer",
    "fields": [
        {"name": "customer_id", "type": "int"},
        {"name": "name", "type": "string"},
        {"name": "email", "type": "string"},
        {"name": "is_active", "type": "boolean"},
        {"name": "balance", "type": "double"},
        {"name": "joined_date", "type": "string"}
    ]
}

# Parse schema
try:
    schema = avro.schema.parse(json.dumps(schema_json))
except Exception as e:
    print(f"❌ Failed to parse schema: {e}")
    raise

# Step 3: Create sample records
records = [
    {"customer_id": 1, "name": "Alice", "email": "alice@example.com", "is_active": True, "balance": 100.0, "joined_date": "2023-01-01"},
    {"customer_id": 2, "name": "Bob", "email": "bob@example.com", "is_active": False, "balance": 150.5, "joined_date": "2023-02-10"},
    # Invalid: missing email
    {"customer_id": 3, "name": "Charlie", "is_active": True, "balance": 50.0, "joined_date": "2023-03-01"},
    # Invalid: string instead of double
    {"customer_id": 4, "name": "Dana", "email": "dana@example.com", "is_active": True, "balance": "NaN", "joined_date": "2023-04-01"},
]

# Step 4: Validate records
valid_records = []
invalid_records = []

for record in records:
    try:
        bytes_writer = io.BytesIO()
        encoder = BinaryEncoder(bytes_writer)
        writer = DatumWriter(schema)
        writer.write(record, encoder)
        valid_records.append(record)
    except Exception as e:
        invalid_records.append((record, str(e)))

# Step 5: Output results
print("✅ Valid Records:")
for r in valid_records:
    print(r)

print("\n❌ Invalid Records:")
for r, e in invalid_records:
    print(f"Record: {r}\nError: {e}\n")

# Step 6: Convert valid records to DataFrame
df = pd.DataFrame(valid_records)
print("\n📘 Final DataFrame (valid records only):")
print(df)

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43994 sha256=42ab02bd0e08bec32f286479cccada1a94bd749c7a348be16738c9a0290b22ce
  Stored in directory: /home/vscode/.cache/pip/wheels/bc/85/62/6cdd81c56f923946b401cecff38055b94c9b766927f7d8ca82
Successfully built avro-python3

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✅ Valid Records:
{'customer_id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'is_active': True, 'balance': 100.0, 'joined_date': '2023-01-01'}
{'customer_id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'is_active': False, 'balance': 150.5, 'joined_date': '2023-02-10'}

❌ Invalid Records:
Record: {'customer_id': 3, 'name': 'Charlie', 'is_active': True, 'balance': 50.0, 'joined_date': '2023-03-01'}
Error: The datum {'customer_id': 3, 'name': 'Charl